In [2]:
text_to_chunk = """
Chères lectrices, chers lecteurs,
Depuis sa fondation en 2001, l'ISI s'est constamment adapté aux évolutions des besoins des formations d'ingénieurs, notamment dans le domaine de l'informatique et de ses applications. Au cours de la dernière décennie, nous avons constaté une augmentation significative des demandes d'admission émanant d'étudiants souhaitant rejoindre nos programmes. Cette tendance positive témoigne de l'intérêt croissant des jeunes talents pour nos cursus.
Soucieux d'innover et de répondre à une diversité de besoins en formation, l'ISI accorde une attention particulière à l'employabilité de ses diplômés et à l'égalité des chances dans l'accès à ses formations. Dans cette perspective, nous prévoyons d'intensifier nos efforts afin d'attirer des candidats de qualité et de sélectionner les meilleurs profils.
Dans une politique de rayonnement, nous nous engageons à renforcer nos partenariats avec des acteurs nationaux et internationaux, qu'ils soient du milieu académique ou industriel. Nous incitons également notre conseil scientifique à établir des collaborations étroites avec nos partenaires socio-économiques.

Conscients de l'importance du développement pédagogique de notre corps enseignant, nous veillons à leur fournir les ressources nécessaires, que ce soit sur le plan matériel, numérique ou financier. Par ailleurs, nous nous efforçons d'évaluer et d'améliorer le niveau de satisfaction des étudiants, des enseignants et du personnel dans toutes nos activités pédagogiques, culturelles et organisationnelles. Dans cette optique, nous avons lancé une campagne de sensibilisation pour promouvoir notre démarche qualité et nous conformer aux normes internationales, notamment en mettant en place un système de management pour les organismes d'éducation selon la norme ISO 21001.
Malgré notre envergure, avec près de 1600 étudiants, nous assumons pleinement notre responsabilité envers tous nos étudiants et les parties prenantes en :
- Offrant une formation adaptée aux exigences du marché de l'emploi et aux besoins spécifiques.
- Respectant les droits de propriété intellectuelle et la vie privée, tout en intégrant nos responsabilités sociales et environnementales ainsi que les besoins spécifiques en matière de formation.
- Proposant un cadre propice à une vie étudiante épanouissante sur les plans culturel, sportif et associatif.
Au nom de tous mes collègues et de l'administration, je tiens à exprimer ma profonde gratitude envers toutes les personnes qui ont contribué, par leur temps et leurs idées, à rehausser le prestige de l'ISI à travers divers projets, que ce soit en matière d'organisation, par la mise en place d'un système SMOE ISO 21001, d'accréditation selon le label CTI-EUR-ACE ou encore par notre participation à des projets nationaux et internationaux.  Nous croyons fermement que notre institution progressera davantage en favorisant une gouvernance transparente et inclusive, qui renforce la coopération, la coordination et la communication.  """

In [31]:

import os
import glob
import PyPDF2
import pandas as pd
import json

current_directory = os.getcwd()

pdf_files = glob.glob(os.path.join(current_directory, 'data/*.pdf'))
csv_files = glob.glob(os.path.join(current_directory, 'data/*.csv'))

def extract_text_from_pdf(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        text = ""
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            text += page.extract_text()
        context = {}
        for key in reader.metadata:
            if key in ["Author","CreationDate","Title"]:
                context[key]= reader.metadata[key];
        context["source"]= file    
        return (str(context),text)

text_dict = {}
for pdf_file in pdf_files:
    (key,text) = extract_text_from_pdf(pdf_file)
    text_dict[key] = text
for csv_file in csv_files:
    with open(csv_file, 'r', encoding='utf-8') as file:
        text_dict[{"source":csv_file }] = file.read()
    


In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

def remove_newlines_without_uppercase(text):
    # Regular expression to match \n followed by a non-uppercase letter or end of string
    return re.sub(r'\n(?![A-Z])', '', text)

def chunkerize(raw):
    # Convert each item in the list to a string
    converted = [remove_newlines_without_uppercase(str(item)) for item in raw]

    # Join all items with a double newline
    text_to_chunk = "".join(converted)

    # Define the text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=50,
        separators=[
            "\n\n",
            "\n",
            ".",
            ",",
            "-",
            " ",
        ]
    )

    # Use the text splitter on the combined string
    return text_splitter.split_text(text_to_chunk)

for file_name in text_dict:
    text_dict[file_name] = chunkerize(text_dict[file_name])

open("output.txt", "w").write(str(text_dict))

1744095